In [1]:
import pandas as pd
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
all_data = pd.concat([train[train.columns[1:]],test[test.columns[1:]]])
all_data.head()
threshold = 20 # Anything that occurs less than this will be removed.
value_counts = all_data.stack().value_counts() # Entire DataFrame 
to_remove = value_counts[value_counts <= threshold].index
all_data_filtered = all_data.replace(to_remove, 0)
all_data_dummies=pd.get_dummies(all_data_filtered, prefix=all_data_filtered.columns,columns=all_data_filtered.columns)
all_data_dummies.head()
X_train = all_data_dummies[:train.shape[0]]
X_test = all_data_dummies[train.shape[0]:]
y = train.ACTION

In [38]:
print test.shape

(58921, 10)


In [48]:
from PNPSKC_clf import PNPSKC_classifier
from sklearn.model_selection import train_test_split
xx_train, xx_test, yy_train, yy_test = train_test_split(X_train, y, test_size=0.5, random_state=42)
# from PNPSKC_clf import PNPSKC_classifier
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
# init a list of classifiers
tuned_params ={}
tuned_params['k'] = [1,2,3]
tuned_params['subsample'] = [0.8]
tuned_params['colsample'] = [0.05,0.06,0.07,0.08,0.09,0.1,0.11]
my_ensemble_clf = []
for my_params in list(ParameterGrid(tuned_params)):
    name = "pnpskc_1_"+str(my_params['k'])+"_"+str(my_params['subsample'])+"_"+str(my_params['colsample'])
    clf = PNPSKC_classifier(k=my_params['k'],subsample=my_params['subsample'],colsample=my_params['colsample'])
    my_ensemble_clf.append((name,clf))

    name = "pnpskc_2_"+str(my_params['k'])+"_"+str(my_params['subsample'])+"_"+str(my_params['colsample'])
    clf = PNPSKC_classifier(k=my_params['k'],subsample=my_params['subsample'],colsample=my_params['colsample'])
    my_ensemble_clf.append((name,clf))

eclf = VotingClassifier(estimators=my_ensemble_clf,voting='soft',n_jobs=-1)
eclf.fit(xx_train,yy_train) 
transformed_xx_test = eclf.transform(xx_test)

0.05
0.05
0.05
0.05
0.05
0.05
0.06
0.06
0.06
0.06
0.06
0.06
0.07
0.07
0.07
0.07
0.07
0.07
0.08
0.08
0.08
0.08
0.08
0.08
0.09
0.09
0.09
0.09
0.09
0.09
0.1
0.1
0.1
0.1
0.1
0.1
0.11
0.11
0.11
0.11
0.11
0.11


/usr/local/lib/python2.7/dist-packages/sklearn/ensemble/voting_classifier.py:290: DeprecationWarning: 'flatten_transform' default value will be changed to True in 0.21.To silence this warning you may explicitly set flatten_transform=False.
  DeprecationWarning)


In [47]:
eclf.score(xx_train,yy_train) 

AxisError: axis 1 is out of bounds for array of dimension 1

In [49]:
import numpy as np
tt = np.transpose(transformed_xx_test)

In [31]:
tt[0:,0]


array([ 0.00793651,  0.        ,  0.        , ...,  0.        ,
        0.        ,  0.        ])

In [39]:
X_test.shape

(58921, 3856)

In [50]:
from sklearn.linear_model import RidgeClassifier
my_linear_clf = RidgeClassifier()
my_linear_clf.fit(tt,yy_test)

X_test_transformed = eclf.transform(X_test)
X_test_transformed_transpose = np.transpose(X_test_transformed)
y_pred = my_linear_clf.decision_function(X_test_transformed_transpose)




/usr/local/lib/python2.7/dist-packages/sklearn/ensemble/voting_classifier.py:290: DeprecationWarning: 'flatten_transform' default value will be changed to True in 0.21.To silence this warning you may explicitly set flatten_transform=False.
  DeprecationWarning)


In [42]:
X_test_transformed.shape

(8, 58921)

In [36]:
len(y_pred)

16385

In [51]:
submission = pd.read_csv("sampleSubmission.csv")
print submission.shape
submission.Action = y_pred
submission.to_csv("submission_SCE1.csv",index = False)

(58921, 2)
